# PII Data NER

## Data
An excel file containing these sheets:
- Export Summary
- PII Train Large Data - PII Trai (800 rows)
- PII Test Data - PII Test Data (16000 rows

## Initial observations/ assumptions:
- The training data is very limited, making the exercise a good candiate for either transfer learning approach or if required semi-supervised learning approach with data augmentation.
- A sentence can have more than one labels, which makes this a multilabel classification problem
- Certain entities such as email, ssn, credit cards, follow a well known pattern. This makes them easier to detect with non-stocastic rule based methods
- Other entities such as names and addresses have more pattern variance and might require a more machine learning centric approach.
- On the other hand the test dataset only requires one label and one PII value, which means this problem forces an assumption that this is one-vs-all classification simplification.
- Additionally, the instructions specifically ask for building a machine learning model, so it' best to skip the regex like rule based approaches since they would be trivial solutions, and not justify the intent of the exercise.
- The problem does not constrain on the compute resources, and does not constrain on the inference time requirements, so it is safe to assume the best bet is to optimize for better accuract / f1- scores.
- The solution should specifically address Entity ambiguity w.r.t overlapping classes (polysemic words) and just the PII, thereby contextual disambiguation needs to be implemented and detailed.

## Solution Space
- I'd treat this problem as a token sequence classification exercise where token is a unit comprising the sequence. 
- Since information is text is contiguous w.r.t token boundaries, it is safe to consider variable token descriptions, such as a token could be a character or a sub word or a word.
- Standard approaches for named entity recorgnition can come in handy, the best expression of training data would be in CONLL format, including part of speed analysis followed IOB tagging using either RNN models or similar approaches.
- Since the exercise is time bound, as a simplification, it is safe to start with off-the-shelf transformer models and then iteratively optimize.

## Evaluation Criteria
There will be a 2 part evaluation
- To determine the performance on a holdout set (with crossvalidation for a real usage, skipped)
  - Report microavg F1 Score, precision and recall for each class
  - A full phrase match might be a good criteria for evaluation.

- To determine perfomance of some known desired behavious with a custom checklist for model explainability and reliability. (Future, for now just test on a small list)
    - Robustness estimates could be included to further harden the models.


In [1]:
# imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle
import random
# local files
import rules
import preprocess
import datagen

from typing import List
from sklearn.model_selection import train_test_split

from flair.data import Sentence, Corpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus
from flair.embeddings import FlairEmbeddings, WordEmbeddings, StackedEmbeddings, TokenEmbeddings,TransformerWordEmbeddings

random.seed(369)

In [2]:
raw_data = pd.read_excel("PII_Train_Large_Data_Test_Data.xlsx",sheet_name="PII Train Large Data - PII Trai", skiprows=1, index_col=None, na_values=['NA'], usecols = "A,B,C")
# shuffle
raw_data = raw_data.sample(frac=1,random_state=369).reset_index(drop=True)

### Data Exploration
Obervations:
- total classes present = 8
- classes are balanced
- 60/20/20 split makes sense for train/test/dev


In [3]:
raw_data.head()

,Text,Labels,PII
0,Quality whom pay travel our 4914569109182 tabl...,CreditCardNumber,4914569109182
1,History effort 427 68 3081 system kitchen. Hea...,SSN,427 68 3081
2,Team Republican him Jessica Ellis reveal. Play...,Name,Jessica Ellis
3,Radio respond perhaps western loss blood. Turn...,Email,jrodriguez@mccarthy-lawson.biz
4,Ready off score 8H 67881 foot market protect.,Plates,8H 67881


In [4]:
# show the distribution of class in the train set
raw_data.groupby(["Labels"])['Text'].nunique().reset_index()

,Labels,Text
0,Address,100
1,CreditCardNumber,100
2,Email,100
3,Name,100
4,None,100
5,Phone_number,100
6,Plates,100
7,SSN,100


## Data Augmentation
- Less support for complex patterns such as <b>Name</b> and <b>Address</b> and those need to be augmented for a fair evaluation
- Lucky for me there is an easy way to generate fake names and addresses, let's add those to the mix to increase support.

In [5]:
# Ref: datagen.py for details
gen = datagen.DataGenerator(max_nb_chars=80, n=900, seed=369)
fake_data = gen.generate_fake_data()
fake_data.head()

,Text,Labels,PII
0,Bring discuss from night Michael Wilson form i...,Name,Michael Wilson
1,About second save despite 42603 Carpenter Corn...,Address,"42603 Carpenter Corner New Jaredburgh, RI 70826"
2,Million natural yet girl. Enough physical Ange...,Name,Angela Wallace
3,"Benefit 5821 Morales Orchard Lorraineport, FL ...",Address,"5821 Morales Orchard Lorraineport, FL 37647"
4,Apply determine Sheila Robinson weight such tr...,Name,Sheila Robinson


In [6]:
# augmented data is a combination of original data and the fake data
aug_data = pd.concat([raw_data, fake_data])
# shuffle again
aug_data = aug_data.sample(frac=1,random_state=369).reset_index(drop=True)

In [8]:
# show the distribution of class in the train set
# Note: Now we have 1000 examples of Name and Address
aug_data.groupby(["Labels"])['Text'].nunique().reset_index()

,Labels,Text
0,Address,1000
1,CreditCardNumber,100
2,Email,100
3,Name,1000
4,None,100
5,Phone_number,100
6,Plates,100
7,SSN,100


## Tokenization
- This is one of the key aspects for getting the correct NER.
- Known patterns can be pinned with deterministic tokens in order to aid the recognition.
- Flair's internal tokenization handles special characters by adding spaces, which makes for special handling in case of email, where "@" symbol must be replaced with " @ " and an inverse transform might be needed on the predicted spans before consumption.

Here is an example of email sentence tokenized, notice tokens <b>eee</b> @ <b>mmm</b>:

In [15]:
# Here is an example of how the tokenization takes place.
converter = preprocess.CoNLLConverter()
index = 61
print(aug_data['Labels'].iloc[index])
converter.bio_tagger(aug_data['Text'].iloc[index], aug_data['Labels'].iloc[index], aug_data['PII'].iloc[index])

Email


['Mission O\n',
 'throughout O\n',
 'small O\n',
 'save O\n',
 'still O\n',
 'lay O\n',
 'discussion O\n',
 'according. O\n',
 'Spend O\n',
 'man O\n',
 'eee O\n',
 'zdawson B-Email\n',
 '@ I-Email\n',
 'hotmail.com I-Email\n',
 'mmm O\n',
 'finally O\n',
 'finish O\n',
 'some O\n',
 'officer. O\n',
 '\n']

In [17]:
# partition data into train, test, dev
X_train, X_test, y_train, y_test = train_test_split(aug_data.index, aug_data["Labels"], test_size=0.2, random_state=369 ,stratify=aug_data["Labels"])
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.25, random_state=369, stratify=y_train)  # 0.25 x 0.8 = 0.2
# verify split distribution is identical
aug_data.loc[X_dev,:].groupby(["Labels"])['Text'].nunique().reset_index()

,Labels,Text
0,Address,200
1,CreditCardNumber,20
2,Email,20
3,Name,200
4,None,20
5,Phone_number,20
6,Plates,20
7,SSN,20


In [18]:
# save all data for visual checks
with open("pii_conll_data_all.txt", "w", encoding="utf-8") as f:
    for index, row in aug_data.iterrows():
        tagged_data = converter.bio_tagger(row['Text'], row['Labels'], row['PII'])
        f.writelines(tagged_data)

In [19]:
# save train.txt
with open("train.txt", "w", encoding="utf-8") as f:
    for index, row in aug_data.loc[X_train,:].iterrows():
        tagged_data = converter.bio_tagger(row['Text'], row['Labels'], row['PII'])
        f.writelines(tagged_data)

In [20]:
# save test.txt
with open("test.txt", "w", encoding="utf-8") as f:
    for index, row in aug_data.loc[X_test,:].iterrows():
        tagged_data = converter.bio_tagger(row['Text'], row['Labels'], row['PII'])
        f.writelines(tagged_data)

In [21]:
# save dev.txt
with open("dev.txt", "w", encoding="utf-8") as f:
    for index, row in aug_data.loc[X_dev,:].iterrows():
        tagged_data = converter.bio_tagger(row['Text'], row['Labels'], row['PII'])
        f.writelines(tagged_data)

In [22]:
#Load the corpus
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '.'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'dev.txt')

2021-01-30 23:20:34,947 Reading data from .
2021-01-30 23:20:34,948 Train: train.txt
2021-01-30 23:20:34,949 Dev: dev.txt
2021-01-30 23:20:34,949 Test: test.txt


In [23]:
# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

# ***
# Note: This would be helpful if we had well formed sentences, this would really help in 
# the case of polysemic words, which we currently don't have in the train / test data
# since that would mean having PER, LOC
# ***

# init multilingual BERT
# bert_embedding = TransformerWordEmbeddings('bert-base-multilingual-cased')

embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        flair_forward_embedding, 
        flair_backward_embedding, 
        # bert_embedding
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

In [24]:
# prepare the tag for ner
tag_type = 'ner'
# make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)

# describe the network
print(tagger)

Dictionary with 16 tags: <unk>, O, B-Address, I-Address, B-Plates, I-Plates, B-Name, I-Name, B-Phone_number, B-SSN, B-CreditCardNumber, B-Email, I-Email, I-SSN, <START>, <STOP>
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(11854, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=11854, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(11854, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=11854, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196

In [25]:
# initialize trainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# start training
trainer.train('./taggers/pii-ner-v0',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2021-01-30 23:21:30,785 ----------------------------------------------------------------------------------------------------
2021-01-30 23:21:30,786 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(11854, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=11854, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(11854, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=11854, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, batch_first=T

{'test_score': 0.998998998998999,
 'dev_score_history': [0.7730727470141152,
  0.8715415019762845,
  0.9346534653465346,
  0.8726554787759132,
  0.9689689689689689,
  0.97,
  0.9760479041916168,
  0.9770687936191426,
  0.9790628115653042,
  0.986986986986987,
  0.983016983016983,
  0.988988988988989,
  0.983016983016983,
  0.9850149850149851,
  0.988,
  0.9899799599198397,
  0.987012987012987,
  0.9830508474576272,
  0.986986986986987,
  0.9850448654037887,
  0.989010989010989,
  0.988,
  0.990990990990991,
  0.99,
  0.990990990990991,
  0.988988988988989,
  0.990990990990991,
  0.9929929929929929,
  0.988988988988989,
  0.990990990990991,
  0.988988988988989,
  0.990990990990991,
  0.9929929929929929,
  0.9929929929929929,
  0.99,
  0.9929929929929929,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.990990990990991,
  0.9

## Scores

In the interest of visibility I have copied over the final scores from the logs.
```
2021-01-30 23:35:08,333 Testing using best model ...
2021-01-30 23:35:08,334 loading file taggers\pii-ner-v0\best-model.pt
2021-01-30 23:35:14,135 1.0000	0.9980	0.9990
2021-01-30 23:35:14,136 
Results:
- F1-score (micro) 0.9990
- F1-score (macro) 0.9996

By class:
Address    tp: 200 - fp: 0 - fn: 0 - precision: 1.0000 - recall: 1.0000 - f1-score: 1.0000
CreditCardNumber tp: 20 - fp: 0 - fn: 0 - precision: 1.0000 - recall: 1.0000 - f1-score: 1.0000
Email      tp: 20 - fp: 0 - fn: 0 - precision: 1.0000 - recall: 1.0000 - f1-score: 1.0000
Name       tp: 199 - fp: 0 - fn: 1 - precision: 1.0000 - recall: 0.9950 - f1-score: 0.9975
Phone_number tp: 20 - fp: 0 - fn: 0 - precision: 1.0000 - recall: 1.0000 - f1-score: 1.0000
Plates     tp: 20 - fp: 0 - fn: 0 - precision: 1.0000 - recall: 1.0000 - f1-score: 1.0000
SSN        tp: 20 - fp: 0 - fn: 0 - precision: 1.0000 - recall: 1.0000 - f1-score: 1.0000
```

In [26]:
# load the trained model
model = SequenceTagger.load('./taggers/pii-ner-v0/final-model.pt')

2021-01-30 23:35:14,154 loading file ./taggers/pii-ner-v0/final-model.pt


In [27]:
# run some basic sanity checks

tests = [
    # name
    ("Hold home fight nor customer defense. Shields Shake player something.","Name"),
    ("Box direction clear sense democratic power. Adult determine rule Deanna across sometimes according.","Name"),
    ("High same pass Smith change. Effort board fly onto middle detail. Ahead chance small ability reduce under. Middle policy use single become.","Name"),
    ("Product similar final sense senior least. Model explain reveal American they behind source. Join Ariana White president other arm until feel force.","Name"),
    ("Matthew Haas Market Republican hand sign.","Name"),
    ("Whom simple Franklin nice three car. Music answer southern performance glass around.","Name"),
    ("Everybody small suggest into president. Over manager Charles government they.","Name"),
    # address
    ("Dark turn purpose attack set way. Significant impact book daughter manager behavior pressure. Price true we pressure culture design 22868 Strong Square Suite 603 serve answer.","Address"),
    ("Class speech structure ask prevent. Do tree actually 384 Moran River Suite 724 West Maureenport, NV 14327 forward.","Address"),
    ("Cold write close full likely actually plant method. Suite 139 Project father chair keep. Financial itself miss house reduce necessary.","Address"),
    ("Cup happen say join improve would. Oil PM special parent executive foot series term. Operation adult result decision prevent talk well. 640 Eddie Mission Apt. 272 New Kimberly, ME 38505","Address"),
    ("Book environmental site each produce more. Clearly 7112 Joanne Cape Apt. 685 ability support technology everything alone pressure.","Address"),
    ("Suite 082 Behavior phone sign meeting service. Fish service attack present during security source.","Address"),
    # credit card
    ("Medical eye inside respond. Ability international change 2224009433974117 could what American actually rate. Whole although chance.","CreditCardNumber"),
    ("Rock movement call turn minute American act soldier. Next customer similar agency 4371942766026400604 training industry. Painting once itself camera go yes.","CreditCardNumber"),
    ("Team deal growth check human. Small these American star subject loss 4504414195582719 measure.","CreditCardNumber"),
    # ssn
    ("Run society adult large. 070-94-9083 Down phone many. Through admit involve life property decision data doctor.","SSN"),
    ("Sister their service tax water sit a. Hundred region bad source well else save. 617 15 2105","SSN"),
    ("Produce military act 765 70 5679 bill. Source carry discussion use such attorney before. Friend air last. Even project mother maintain allow partner.","SSN"),
    ("573-70-8827 Man open upon. Activity note area behavior. Page none two morning.","SSN"),
    # email
    ("Discover our nothing try chance federal candidate. Indeed design barbaracarpenter@yahoo.com high people.","Email"),
    ("Magazine day physical. Eight fill on whatever glass technology. Act as pass tell action pattern huertabruce@yahoo.com every.","Email"),
    ("hensleyedward@gmail.com Increase several him marriage word truth sort. Among economic system. Usually serve candidate moment.","Email"),
    ("Lose meet left kitchen organization herself. Business majority our. They job watch reason meet emily86@gmail.com drive.","Email"),
    ("Save fall card care anyone pressure. Remember susan68@ramirez.biz sister offer draw. Take field because return fear guess.","Email"),
    ("General attorney record ten much remember. Eye live ask bridget58@gmail.com support.","Email"),
    # plate
    ("Trouble religious hear environmental teacher get ZSH 214 forget. Test executive whole pass design four contain.","Plate"),
    ("Draw 40LR5 feeling might. Under lawyer part senior.","Plate"),
    ("Able this character evidence woman it want. I require popular BAA0043 off maintain.","Plate"),
    ("Of staff international box would also. Arrive region actually training. 255-519","Plate"),
    ("Population religious effort whatever contain. Employee general wife another thus mission others. 22-C881 Story year owner easy common late. Before build not gun pay upon.","Plate"),
    ("Whose raise soon participant S52 1IO alone. Value design teacher which.","Plate"),
    # none
    ("Ball idea ever move. Sign writer third teach drop perhaps sometimes. Explain happy me good enjoy.","None"),
    ("Usually cold thousand professional TV direction care. Carry but role strong sister few. Region base single investment.","None"),
    ("Old start difficult others. Move decide shake talk million candidate.","None"),
    ("Manager physical respond that teach father together. Republican in recently ahead.","None"),
    ("Couple job civil green road news senior. Occur put standard air collection actually thus. Single country would them.","None"),

]

engine =rules.RulesEngine()

for text, true_label in tests:
    text = engine.pin_text(text)
    sentence = Sentence(text)
    # predict the tags
    model.predict(sentence)
    print("Sentence:")
    print(text)
    print("Tagged Sentence:")
    print(sentence.to_tagged_string())
    print("True Label:", true_label)
    print("Entity")
    for entity in sentence.get_spans('ner'):
        print(entity)
    print("\n")

Sentence:
Hold home fight nor customer defense. Shields Shake player something.
Tagged Sentence:
Hold home fight nor customer defense . Shields <B-Name> Shake player something .
True Label: Name
Entity
Span [8]: "Shields"   [− Labels: Name (0.9508)]


Sentence:
Box direction clear sense democratic power. Adult determine rule Deanna across sometimes according.
Tagged Sentence:
Box direction clear sense democratic power . Adult determine rule Deanna <B-Name> across sometimes according .
True Label: Name
Entity
Span [11]: "Deanna"   [− Labels: Name (0.997)]


Sentence:
High same pass Smith change. Effort board fly onto middle detail. Ahead chance small ability reduce under. Middle policy use single become.
Tagged Sentence:
High same pass Smith <B-Name> change . Effort board fly onto middle detail . Ahead chance small ability reduce under . Middle policy use single become .
True Label: Name
Entity
Span [4]: "Smith"   [− Labels: Name (0.9831)]


Sentence:
Product similar final sense senior 